In [1]:
%run nlp-functions.ipynb

2024-05-21 14:06:02.671955: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-05-21 14:06:02.671975: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-05-21 14:06:02.671978: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-05-21 14:06:02.671991: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-21 14:06:02.671999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
#train = pd.read_json('data/train.json')
#test = pd.read_json('data/test.json')

In [2]:
df1 = pd.read_json('initial_predictions.json')

In [5]:
df1.head()

,document,token,token_text,correct_label,preds,cm,sentence_text,tokenized_sentence,labels_in_sentence
0,7,0,Design,O,O,TN,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,7,1,Thinking,O,O,TN,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
2,7,2,for,O,O,TN,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
3,7,3,innovation,O,O,TN,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
4,7,4,reflexion,O,O,TN,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."


The dataset df1 contains the results of the Presidio Analyzer Function on each individual token in Learning Agency PII dataset. Each row contains the document number, word token number, word token text, correct label, Presidio Analyzer prediction, and classification as True Negative, True Positive, False Positive or False Negative for the prediction. We also included three columns containing the sentence context of each word.

In the next section, we are going to divide this initial dataset into negative and positive labels. We will create a more balanced dataset to help the model learn better.

In [58]:
start = time.time()
sample_35, test_idx_35 = create_sample_data(df1, 0.3, 3000, 35)
end = time.time()
print(end - start)

17.172152042388916


In [168]:
record_df = RecordDataFrame()

In [91]:
#Round 1
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 5,
     'embed_dim' : 32,
     'Num Transformer Heads' : 4,
     'ff_dim' : 64,
     'Batch Size' : 32}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/5


2024-05-20 00:18:40.483109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


98/98 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 0.6368
Epoch 2/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.1342
Epoch 3/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.1098
Epoch 4/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.0434
Epoch 5/5
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.0246


In [160]:
results_35, record_dict = calculate_metrics(val_dataset, ner_model)
results_35

processed 15562 tokens
True Positive: 114, False Positive: 258, False Negative: 236
Weighted Precision: 0.31695989067563785
Weighted Recall: 0.32571428571428573
Weighted F1: 0.3019689473552832
Micro F5: 0.3249287436965578


2024-05-20 02:39:06.037103: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.445378,0.301136,0.359322,176,53,66,123
1,I-NAME_STUDENT,0.241107,0.451852,0.314433,135,61,192,74
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [172]:
#Round 2: Change the model architecture - increase dimensions and decrease batch size
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 5,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - loss: 0.2774
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0492
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 0.0215
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.0156
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 0.0112


In [173]:
results_35, record_dict = calculate_metrics(val_dataset, ner_model)
results_35
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)

processed 15562 tokens
True Positive: 192, False Positive: 325, False Negative: 158
Weighted Precision: 0.37083783072265364
Weighted Recall: 0.5485714285714286
Weighted F1: 0.4353723412625561
Micro F5: 0.5386856587892522


2024-05-20 02:55:29.749645: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [175]:
results_35

,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.518692,0.630682,0.569231,176,111,103,65
1,I-NAME_STUDENT,0.285211,0.600000,0.386635,135,81,203,54
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,3,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,16,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [176]:
#Round 3: See how these new dimensions do with LSTM
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 5,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 20s 88ms/step - loss: 0.2937
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 17s 86ms/step - loss: 0.0915
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 16s 84ms/step - loss: 0.0780
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 17s 87ms/step - loss: 0.0157
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - loss: 0.0061


In [177]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Model Description': 'NER Transformer With Bidirectional LSTM'})
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 136, False Positive: 30, False Negative: 214
Weighted Precision: 0.7532491248926754
Weighted Recall: 0.3885714285714286
Weighted F1: 0.5088940467477877
Micro F5: 0.3965903992821893


2024-05-20 03:02:50.439351: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.864078,0.505682,0.637993,176,89,14,87
1,I-NAME_STUDENT,0.821429,0.340741,0.481675,135,46,10,89
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,1,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [186]:
#Round 4: Further finetune the NER without LSTM
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 5,
     'embed_dim' : 64,
     'Num Transformer Heads' : 6,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - loss: 0.3006
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.0869
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0317
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 0.0274
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 0.0280


In [187]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 176, False Positive: 327, False Negative: 174
Weighted Precision: 0.4482776908954919
Weighted Recall: 0.5028571428571429
Weighted F1: 0.42550817189115064
Micro F5: 0.4945423106019669


2024-05-20 03:18:19.151276: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.710744,0.488636,0.579125,176,86,35,90
1,I-NAME_STUDENT,0.235602,0.666667,0.348162,135,90,292,45
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [189]:
#Round 5: Try NER model with LSTM and fewer heads
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 5,
     'embed_dim' : 64,
     'Num Transformer Heads' : 2,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 23s 103ms/step - loss: 0.2518
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - loss: 0.0450
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - loss: 0.0096
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - loss: 0.0037
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 91ms/step - loss: 0.0013


In [190]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 128, False Positive: 29, False Negative: 222
Weighted Precision: 0.7504949104949103
Weighted Recall: 0.3657142857142857
Weighted F1: 0.4810638903333509
Micro F5: 0.3736387111260806


2024-05-20 03:23:04.752772: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.783784,0.494318,0.606272,176,87,24,89
1,I-NAME_STUDENT,0.909091,0.296296,0.446927,135,40,4,95
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [192]:
#Round 6: Try NER model with LSTM with fewer epochs. Model seemed a bit overfit last time
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 2,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 115ms/step - loss: 0.2350
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - loss: 0.0446
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 19s 96ms/step - loss: 0.0088


In [193]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 140, False Positive: 20, False Negative: 210
Weighted Precision: 0.8107332738626227
Weighted Recall: 0.4
Weighted F1: 0.5344147297143074
Micro F5: 0.40852974186307517


2024-05-20 03:26:19.661344: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.884211,0.477273,0.619926,176,84,11,92
1,I-NAME_STUDENT,0.949153,0.414815,0.577320,135,56,3,79
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,5,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,1,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


Decreasing the number of heads and epochs increased the precision quite a bit. I'm going to see if simplifying the bidirectional LSTM architecture and decreasing batch size will improve the model.

In [195]:
#Round 7: Try NER model with LSTM with smaller batch size and less dimensions
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 93ms/step - loss: 0.4362
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - loss: 0.0541
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 83ms/step - loss: 0.0217


In [196]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 196, False Positive: 109, False Negative: 154
Weighted Precision: 0.5778100491575668
Weighted Recall: 0.56
Weighted F1: 0.5682175015619831
Micro F5: 0.5627829928216455


2024-05-20 03:32:19.844561: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.697531,0.642045,0.668639,176,113,49,63
1,I-NAME_STUDENT,0.588652,0.614815,0.601449,135,83,58,52
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,1,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,1,0


In [197]:
record_df

,Model Description,Weighted Precision,Weighted Recall,Weighted F1,Micro F5,True Positive,False Negative,False Positive,Sample Size,Epochs,Sample Random State,Batch Size,Num Transformer Heads,ff_dim,embed_dim,Dataset Type
0,NER Transformer Without LSTM,0.31696,0.325714,0.301969,0.324929,114,236,258,15562,5,35,32,4,64,32,validation
1,NER Transformer Without LSTM,0.370838,0.548571,0.435372,0.538686,192,158,325,15562,5,35,16,4,128,64,validation
2,NER Transformer With Bidirectional LSTM,0.753249,0.388571,0.508894,0.39659,136,214,30,15562,5,35,16,4,128,64,validation
3,NER Transformer Without LSTM,0.448278,0.502857,0.425508,0.494542,176,174,327,15562,5,35,16,6,128,64,validation
4,NER Transformer With Bidirectional LSTM,0.750495,0.365714,0.481064,0.373639,128,222,29,15562,5,35,16,2,128,64,validation
5,NER Transformer With Bidirectional LSTM,0.810733,0.4,0.534415,0.40853,140,210,20,15562,3,35,16,2,128,64,validation
6,NER Transformer With Bidirectional LSTM,0.57781,0.56,0.568218,0.562783,196,154,109,15562,3,35,8,2,32,16,validation


A simpler architecture and smaller batch size did improve the bidirectional LSTM to surpass the previous Micro F5 high score from network without an LSTM!

In [198]:
#Round 8: Perhaps we made the network too simple. Try increasing the dimensions a bit
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 97ms/step - loss: 0.2928
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 34s 86ms/step - loss: 0.0402
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 34s 87ms/step - loss: 0.0101


In [199]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 159, False Positive: 55, False Negative: 191
Weighted Precision: 0.6928591242322171
Weighted Recall: 0.4542857142857143
Weighted F1: 0.5462096774193548
Micro F5: 0.46117804551539493


2024-05-20 03:40:55.051578: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.873786,0.511364,0.645161,176,90,13,86
1,I-NAME_STUDENT,0.657143,0.511111,0.575000,135,69,36,66
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,6,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


So for the Bidirectional LSTM, smaller architecture helps to balance high precision and low recall, resulting in a better micro f5 score. What about if we decreased the epochs, number of heads, and batch size of the network without an LSTM?

In [200]:
#Round 9: Try to decrease batch size and see if that helps the network without the LSTM
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 2,
     'ff_dim' : 128,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - loss: 0.3044
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - loss: 0.0511
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - loss: 0.0387


In [201]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 142, False Positive: 130, False Negative: 208
Weighted Precision: 0.506492973237199
Weighted Recall: 0.4057142857142857
Weighted F1: 0.4364998636441685
Micro F5: 0.4092219020172911


2024-05-20 03:49:15.243702: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.685714,0.409091,0.512456,176,72,33,104
1,I-NAME_STUDENT,0.419162,0.518519,0.463576,135,70,97,65
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [202]:
#Round 10: Increase heads on this transformer
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - loss: 0.2377
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - loss: 0.0427
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - loss: 0.0217


In [203]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 148, False Positive: 161, False Negative: 202
Weighted Precision: 0.42213583905177793
Weighted Recall: 0.4228571428571429
Weighted F1: 0.40839768672326815
Micro F5: 0.42477094602053206


2024-05-20 03:53:19.402255: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.484716,0.630682,0.548148,176,111,118,65
1,I-NAME_STUDENT,0.462500,0.274074,0.344186,135,37,43,98
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [205]:
#Round 10: Try again with the LSTM decreasing batch size
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
781/781 ━━━━━━━━━━━━━━━━━━━━ 73s 88ms/step - loss: 0.3329
Epoch 2/3
781/781 ━━━━━━━━━━━━━━━━━━━━ 65s 83ms/step - loss: 0.0481
Epoch 3/3
781/781 ━━━━━━━━━━━━━━━━━━━━ 67s 85ms/step - loss: 0.0187


In [206]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 142, False Positive: 23, False Negative: 208
Weighted Precision: 0.7506097442380628
Weighted Recall: 0.4057142857142857
Weighted F1: 0.5199694423223835
Micro F5: 0.41413348289399887


2024-05-20 04:03:30.109773: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.902655,0.579545,0.705882,176,102,11,74
1,I-NAME_STUDENT,0.769231,0.296296,0.427807,135,40,12,95
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [209]:
#Round 11: What happens if we use just an LSTM without the Bidirectional component? Try it on the best params for Bidirectional LSTM
r = {'Model Description' : 'NER Transformer With One Direction LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithOneLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - loss: 0.4177
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 40ms/step - loss: 0.0547
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - loss: 0.0201


In [210]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 194, False Positive: 120, False Negative: 156
Weighted Precision: 0.5567125927353309
Weighted Recall: 0.5542857142857143
Weighted F1: 0.5465532398635847
Micro F5: 0.5564872021182701


2024-05-20 04:08:57.642833: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.616822,0.750000,0.676923,176,132,82,44
1,I-NAME_STUDENT,0.639175,0.459259,0.534483,135,62,35,73
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,3,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [211]:
#Round 12: Try increasing number dimensions of NER model without LSTM
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 128,
     'Num Transformer Heads' : 4,
     'ff_dim' : 256,
     'Batch Size' : 32}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 17s 131ms/step - loss: 0.3721
Epoch 2/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - loss: 0.1200
Epoch 3/3
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - loss: 0.0391


In [212]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 154, False Positive: 326, False Negative: 196
Weighted Precision: 0.30959986285464924
Weighted Recall: 0.43999999999999995
Weighted F1: 0.3534900361070532
Micro F5: 0.43380281690140843


2024-05-20 04:11:53.233240: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.417989,0.448864,0.432877,176,79,110,97
1,I-NAME_STUDENT,0.257732,0.555556,0.352113,135,75,216,60
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [214]:
#Round 13: Try increasing number dimensions of NER model with one LSTM
r = {'Model Description' : 'NER Transformer With One Direction LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 4,
     'ff_dim' : 64,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithOneLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 22s 84ms/step - loss: 0.3009
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - loss: 0.0690
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 0.0193


In [215]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 172, False Positive: 90, False Negative: 178
Weighted Precision: 0.6344453707949477
Weighted Recall: 0.49142857142857144
Weighted F1: 0.5523095899695019
Micro F5: 0.49622725255215266


2024-05-20 04:15:05.608146: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.711409,0.602273,0.652308,176,106,43,70
1,I-NAME_STUDENT,0.717391,0.488889,0.581498,135,66,26,69
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,2,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,1,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,13,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,2,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [216]:
#Round 14: Trying to reproduce max results of the Bidrectional LSTM. Who knows what can happen with gradient descent?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 51s 116ms/step - loss: 0.4844
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 95ms/step - loss: 0.0574
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 97ms/step - loss: 0.0208


In [217]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 165, False Positive: 91, False Negative: 185
Weighted Precision: 0.6137755240288773
Weighted Recall: 0.4714285714285714
Weighted F1: 0.5311896272770853
Micro F5: 0.4763491005996003


2024-05-20 04:23:29.782153: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.770492,0.534091,0.630872,176,94,28,82
1,I-NAME_STUDENT,0.586777,0.525926,0.554688,135,71,50,64
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,7,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,3,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,1,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [218]:
#Round 15: Simplifying the network and increasing number of heads
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 8,
     'Num Transformer Heads' : 4,
     'ff_dim' : 16,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 53s 116ms/step - loss: 1.2906
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 96ms/step - loss: 0.1097
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 97ms/step - loss: 0.0474


In [219]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 120, False Positive: 72, False Negative: 230
Weighted Precision: 0.5700000000000001
Weighted Recall: 0.34285714285714286
Weighted F1: 0.42651533340608966
Micro F5: 0.3489152314918363


2024-05-20 04:27:16.894361: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.75,0.409091,0.529412,176,72,24,104
1,I-NAME_STUDENT,0.50,0.355556,0.415584,135,48,48,87
2,B-EMAIL,0.00,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.00,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.00,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.00,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.00,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.00,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.00,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.00,0.000000,0.000000,0,0,0,0


Simplifying the network and increasing the number of transformers causes a much lower recall.

In [220]:
#Round 17: Trying to replicate the best params for bidirectional LSTM - was it a fluke?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 52s 119ms/step - loss: 0.6234
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 97ms/step - loss: 0.0718
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 39s 100ms/step - loss: 0.0261


In [221]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 110, False Positive: 26, False Negative: 240
Weighted Precision: 0.7235792778649921
Weighted Recall: 0.3142857142857143
Weighted F1: 0.43406099518459074
Micro F5: 0.3218546027458924


2024-05-20 04:34:10.314784: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.791209,0.409091,0.539326,176,72,19,104
1,I-NAME_STUDENT,0.844444,0.281481,0.422222,135,38,7,97
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [222]:
#Round 18: Trying to replicate the best params for bidirectional LSTM - was it a fluke?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 53s 119ms/step - loss: 0.5074
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 98ms/step - loss: 0.0598
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - loss: 0.0227


In [223]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 111, False Positive: 36, False Negative: 239
Weighted Precision: 0.6778584037321901
Weighted Recall: 0.31714285714285717
Weighted F1: 0.4247960021396676
Micro F5: 0.3243790041587052


2024-05-20 04:38:51.749641: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.737864,0.431818,0.544803,176,76,27,100
1,I-NAME_STUDENT,0.795455,0.259259,0.391061,135,35,9,100
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [224]:
#Round 19: Trying to replicate the best params for bidirectional LSTM - was it a fluke?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 126ms/step - loss: 0.2705
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 101ms/step - loss: 0.0750
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - loss: 0.0182


In [225]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 205, False Positive: 58, False Negative: 145
Weighted Precision: 0.6997916702895401
Weighted Recall: 0.5857142857142857
Weighted F1: 0.6371776352666652
Micro F5: 0.5913680239653833


2024-05-20 04:43:24.373034: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.754839,0.664773,0.706949,176,117,38,59
1,I-NAME_STUDENT,0.830189,0.651852,0.730290,135,88,18,47
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,2,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [226]:
#Round 20: Trying to replicate the best params for bidirectional LSTM - was it a fluke?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 128ms/step - loss: 0.2979
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - loss: 0.0451
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 105ms/step - loss: 0.0129


In [227]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 172, False Positive: 93, False Negative: 178
Weighted Precision: 0.6256008057747188
Weighted Recall: 0.4914285714285714
Weighted F1: 0.5339629258905778
Micro F5: 0.4960621186910704


2024-05-20 04:46:53.622364: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.632432,0.664773,0.648199,176,117,68,59
1,I-NAME_STUDENT,0.782609,0.400000,0.529412,135,54,15,81
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,2,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,5,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,2,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [228]:
#Round 21: Trying to replicate the best params for bidirectional LSTM - was it a fluke?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 59s 136ms/step - loss: 0.3625
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 44s 112ms/step - loss: 0.0494
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 43s 110ms/step - loss: 0.0137


In [229]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 167, False Positive: 61, False Negative: 183
Weighted Precision: 0.6643832802711307
Weighted Recall: 0.4771428571428571
Weighted F1: 0.554377022295111
Micro F5: 0.4836266429048786


2024-05-20 04:51:03.486211: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.769231,0.511364,0.614334,176,90,27,86
1,I-NAME_STUDENT,0.719626,0.570370,0.636364,135,77,30,58
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,1,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,3,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [230]:
#Round 22: Trying to replicate the best params for bidirectional LSTM - increase batch size
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 46s 195ms/step - loss: 0.3640
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 127ms/step - loss: 0.0725
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 127ms/step - loss: 0.0224


In [231]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 135, False Positive: 81, False Negative: 215
Weighted Precision: 0.5865385741007011
Weighted Recall: 0.3857142857142857
Weighted F1: 0.4627075621364358
Micro F5: 0.39147892036582643


2024-05-20 04:54:23.052608: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.786408,0.460227,0.580645,176,81,22,95
1,I-NAME_STUDENT,0.495413,0.400000,0.442623,135,54,55,81
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,3,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,1,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [232]:
#Round 23: Trying to replicate the best params for bidirectional LSTM - increase batch size
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 35,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 4,
     'ff_dim' : 64,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = sample_35, test_idx_35 #create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 48s 201ms/step - loss: 0.3650
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 131ms/step - loss: 0.0859
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 25s 127ms/step - loss: 0.0298


In [233]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15562 tokens
True Positive: 130, False Positive: 22, False Negative: 220
Weighted Precision: 0.7522689075630251
Weighted Recall: 0.3714285714285714
Weighted F1: 0.49238632260215
Micro F5: 0.3796899573129634


2024-05-20 04:57:33.038895: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.882353,0.511364,0.647482,176,90,12,86
1,I-NAME_STUDENT,0.800000,0.296296,0.432432,135,40,10,95
2,B-EMAIL,0.000000,0.000000,0.000000,4,0,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,23,0,0,23
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,6,0,0,6
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,1,0,0,1
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [234]:
#Round 23: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 70s 155ms/step - loss: 0.4576
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 46s 118ms/step - loss: 0.0736
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 47s 119ms/step - loss: 0.0297


In [235]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

2024-05-20 05:06:05.224776: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


processed 15952 tokens
True Positive: 141, False Positive: 79, False Negative: 215
Weighted Precision: 0.5851195856139402
Weighted Recall: 0.3960674157303371
Weighted F1: 0.46284455773425126
Micro F5: 0.4019736842105263


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.620253,0.526882,0.569767,186,98,60,88
1,I-NAME_STUDENT,0.693548,0.320896,0.438776,134,43,19,91
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [238]:
#Round 25: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 78s 170ms/step - loss: 0.2586
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 49s 124ms/step - loss: 0.0493
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 50s 129ms/step - loss: 0.0144


In [239]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 185, False Positive: 105, False Negative: 171
Weighted Precision: 0.6323743100254586
Weighted Recall: 0.5196629213483146
Weighted F1: 0.5667527556475719
Micro F5: 0.5233949945593036


2024-05-20 05:13:52.418463: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.630058,0.586022,0.607242,186,109,64,77
1,I-NAME_STUDENT,0.778947,0.552239,0.646288,134,74,21,60
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.055556,0.100000,0.071429,10,1,17,9
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.250000,0.083333,0.125000,12,1,3,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [241]:
#Round 26: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 54s 225ms/step - loss: 0.3786
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 133ms/step - loss: 0.0843
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 27s 136ms/step - loss: 0.0289


In [242]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 139, False Positive: 32, False Negative: 217
Weighted Precision: 0.7270153379703941
Weighted Recall: 0.3904494382022472
Weighted F1: 0.5068605323838313
Micro F5: 0.3984125234263036


2024-05-20 12:11:37.608482: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.842593,0.489247,0.619048,186,91,17,95
1,I-NAME_STUDENT,0.761905,0.358209,0.487310,134,48,15,86
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [243]:
#Round 27: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 80s 179ms/step - loss: 0.2778
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 50s 127ms/step - loss: 0.0532
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 50s 129ms/step - loss: 0.0175


In [244]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 204, False Positive: 75, False Negative: 152
Weighted Precision: 0.6877271218195352
Weighted Recall: 0.5730337078651685
Weighted F1: 0.6242791574973598
Micro F5: 0.5778407233903475


2024-05-20 12:20:13.236346: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.748466,0.655914,0.699140,186,122,41,64
1,I-NAME_STUDENT,0.776699,0.597015,0.675105,134,80,23,54
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.153846,0.200000,0.173913,10,2,11,8
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [245]:
#Round 28: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 79s 175ms/step - loss: 0.5850
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 49s 126ms/step - loss: 0.1224
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 49s 126ms/step - loss: 0.0947


In [246]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 181, False Positive: 129, False Negative: 175
Weighted Precision: 0.5311540696808035
Weighted Recall: 0.5084269662921348
Weighted F1: 0.5029294428276833
Micro F5: 0.5109663409337677


2024-05-20 12:27:34.506313: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.573276,0.715054,0.636364,186,133,99,53
1,I-NAME_STUDENT,0.615385,0.358209,0.452830,134,48,30,86
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [247]:
#Round 29: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With One LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithOneLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 44s 86ms/step - loss: 0.2520
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - loss: 0.0477
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - loss: 0.0151


In [248]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 168, False Positive: 89, False Negative: 188
Weighted Precision: 0.6577383986894784
Weighted Recall: 0.4719101123595505
Weighted F1: 0.5328276124624438
Micro F5: 0.4770121218739762


2024-05-20 12:34:47.219753: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.597701,0.559140,0.577778,186,104,70,82
1,I-NAME_STUDENT,0.815789,0.462687,0.590476,134,62,14,72
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.166667,0.100000,0.125000,10,1,5,9
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.000000,0.083333,0.153846,12,1,0,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [249]:
#Round 30: Test best params on different datasets
r = {'Model Description' : 'NER Transformer With One LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 2,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithOneLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 45s 88ms/step - loss: 0.5148
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - loss: 0.0834
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step - loss: 0.0315


In [250]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 123, False Positive: 105, False Negative: 233
Weighted Precision: 0.49002088735234794
Weighted Recall: 0.3455056179775281
Weighted F1: 0.4052515876893015
Micro F5: 0.35035056967572303


2024-05-20 12:37:06.181405: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.607692,0.424731,0.500000,186,79,51,107
1,I-NAME_STUDENT,0.458333,0.328358,0.382609,134,44,52,90
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,2,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [251]:
record_df

,Model Description,Weighted Precision,Weighted Recall,Weighted F1,Micro F5,True Positive,False Negative,False Positive,Sample Size,Epochs,Sample Random State,Batch Size,Num Transformer Heads,ff_dim,embed_dim,Dataset Type
0,NER Transformer Without LSTM,0.31696,0.325714,0.301969,0.324929,114,236,258,15562,5,35,32,4,64,32,validation
1,NER Transformer Without LSTM,0.370838,0.548571,0.435372,0.538686,192,158,325,15562,5,35,16,4,128,64,validation
2,NER Transformer With Bidirectional LSTM,0.753249,0.388571,0.508894,0.39659,136,214,30,15562,5,35,16,4,128,64,validation
3,NER Transformer Without LSTM,0.448278,0.502857,0.425508,0.494542,176,174,327,15562,5,35,16,6,128,64,validation
4,NER Transformer With Bidirectional LSTM,0.750495,0.365714,0.481064,0.373639,128,222,29,15562,5,35,16,2,128,64,validation
5,NER Transformer With Bidirectional LSTM,0.810733,0.4,0.534415,0.40853,140,210,20,15562,3,35,16,2,128,64,validation
6,NER Transformer With Bidirectional LSTM,0.57781,0.56,0.568218,0.562783,196,154,109,15562,3,35,8,2,32,16,validation
7,NER Transformer With Bidirectional LSTM,0.692859,0.454286,0.54621,0.461178,159,191,55,15562,3,35,8,2,64,32,validation
8,NER Transformer Without LSTM,0.506493,0.405714,0.4365,0.409222,142,208,130,15562,3,35,8,2,128,64,validation
9,NER Transformer Without LSTM,0.422136,0.422857,0.408398,0.424771,148,202,161,15562,3,35,8,4,128,64,validation


In [252]:
#Round 31: Test best params on different datasets
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 39s 146ms/step - loss: 0.2955
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.0662
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.0296


In [253]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 172, False Positive: 515, False Negative: 184
Weighted Precision: 0.2776517841995664
Weighted Recall: 0.4831460674157303
Weighted F1: 0.33947877652933833
Micro F5: 0.46646500469385627


2024-05-20 12:42:20.434880: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.418699,0.553763,0.476852,186,103,143,83
1,I-NAME_STUDENT,0.156463,0.514925,0.240000,134,69,372,65
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [254]:
#Round 32: Test best params on different datasets
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 37s 141ms/step - loss: 0.2792
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.0620
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0289


In [255]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 165, False Positive: 545, False Negative: 191
Weighted Precision: 0.2735946524250917
Weighted Recall: 0.4634831460674157
Weighted F1: 0.3301928919772134
Micro F5: 0.4464099895941727


2024-05-20 12:43:53.329309: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.429752,0.559140,0.485981,186,104,138,82
1,I-NAME_STUDENT,0.130342,0.455224,0.202658,134,61,407,73
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [256]:
#Round 33: Test best params on different datasets
r = {'Model Description' : 'NER Transformer Without LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 64,
     'Num Transformer Heads' : 4,
     'ff_dim' : 128,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModel(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 37s 143ms/step - loss: 0.3510
Epoch 2/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0917
Epoch 3/3
196/196 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 0.0324


In [257]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 170, False Positive: 400, False Negative: 186
Weighted Precision: 0.3229735809336793
Weighted Recall: 0.4775280898876405
Weighted F1: 0.3710005760923127
Micro F5: 0.46673706441393875


2024-05-20 12:45:23.560488: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.478469,0.537634,0.506329,186,100,109,86
1,I-NAME_STUDENT,0.193906,0.522388,0.282828,134,70,291,64
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [258]:
#Round 34: Further finetuning bidirectional LSTM?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 95s 208ms/step - loss: 0.2664
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 144ms/step - loss: 0.0502
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 58s 148ms/step - loss: 0.0163


In [259]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 184, False Positive: 45, False Negative: 172
Weighted Precision: 0.7593406006985209
Weighted Recall: 0.5168539325842696
Weighted F1: 0.6058921210332556
Micro F5: 0.5240442545733377


2024-05-20 12:52:20.040325: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.760563,0.580645,0.658537,186,108,34,78
1,I-NAME_STUDENT,0.872093,0.559701,0.681818,134,75,11,59
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.000000,0.083333,0.153846,12,1,0,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [260]:
#Round 35: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 95s 211ms/step - loss: 0.3103
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 58s 149ms/step - loss: 0.0540
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - loss: 0.0159


In [261]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 233, False Positive: 91, False Negative: 123
Weighted Precision: 0.6859769444039105
Weighted Recall: 0.6544943820224719
Weighted F1: 0.6548663095511315
Micro F5: 0.656764960971379


2024-05-20 12:57:13.223053: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.761905,0.688172,0.723164,186,128,40,58
1,I-NAME_STUDENT,0.675325,0.776119,0.722222,134,104,50,30
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,1,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.000000,0.083333,0.153846,12,1,0,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [262]:
#Round 36: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 96s 216ms/step - loss: 0.3668
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 61s 155ms/step - loss: 0.0503
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 60s 154ms/step - loss: 0.0170


In [263]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 150, False Positive: 50, False Negative: 206
Weighted Precision: 0.7227291935964999
Weighted Recall: 0.4213483146067416
Weighted F1: 0.5281947460517422
Micro F5: 0.42857142857142855


2024-05-20 13:03:52.972073: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.79646,0.483871,0.602007,186,90,23,96
1,I-NAME_STUDENT,0.72500,0.432836,0.542056,134,58,22,76
2,B-EMAIL,0.00000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.00000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.00000,0.000000,0.000000,10,0,5,10
5,I-URL_PERSONAL,0.00000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.00000,0.166667,0.285714,12,2,0,10
7,I-ID_NUM,0.00000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.00000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.00000,0.000000,0.000000,0,0,0,0


In [264]:
#Round 37: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 1,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 101s 227ms/step - loss: 0.3271
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 61s 157ms/step - loss: 0.0838
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 61s 155ms/step - loss: 0.0303


In [265]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 188, False Positive: 92, False Negative: 168
Weighted Precision: 0.6157849563046192
Weighted Recall: 0.5280898876404494
Weighted F1: 0.5632795873980223
Micro F5: 0.5324618736383442


2024-05-20 13:08:16.939543: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.733333,0.532258,0.616822,186,99,36,87
1,I-NAME_STUDENT,0.618056,0.664179,0.640288,134,89,55,45
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,1,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [266]:
#Round 38: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 16,
     'Num Transformer Heads' : 1,
     'ff_dim' : 32,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 108s 240ms/step - loss: 0.3543
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 65s 167ms/step - loss: 0.0652
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - loss: 0.0267


In [267]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 146, False Positive: 97, False Negative: 210
Weighted Precision: 0.5697363844015072
Weighted Recall: 0.41011235955056174
Weighted F1: 0.4675272391654102
Micro F5: 0.4151810127966751


2024-05-20 13:13:32.568222: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.630872,0.505376,0.561194,186,94,55,92
1,I-NAME_STUDENT,0.548387,0.380597,0.449339,134,51,42,83
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,0,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.000000,0.083333,0.153846,12,1,0,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [268]:
#Round 39: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 24,
     'Num Transformer Heads' : 1,
     'ff_dim' : 48,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 114s 251ms/step - loss: 0.4045
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 64s 163ms/step - loss: 0.0651
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 69s 178ms/step - loss: 0.0222


In [269]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 188, False Positive: 73, False Negative: 168
Weighted Precision: 0.6502803890780807
Weighted Recall: 0.5280898876404494
Weighted F1: 0.5825759475443728
Micro F5: 0.5335662045628207


2024-05-20 13:19:44.148013: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.675862,0.526882,0.592145,186,98,47,88
1,I-NAME_STUDENT,0.789474,0.671642,0.725806,134,90,24,44
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,2,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [270]:
#Round 40: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 24,
     'Num Transformer Heads' : 1,
     'ff_dim' : 48,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 111s 246ms/step - loss: 0.2676
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 184ms/step - loss: 0.0501
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 70s 178ms/step - loss: 0.0185


In [271]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 144, False Positive: 44, False Negative: 212
Weighted Precision: 0.7076696711528172
Weighted Recall: 0.4044943820224719
Weighted F1: 0.5147323018955346
Micro F5: 0.4119718309859155


2024-05-20 13:25:01.358445: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.809524,0.456989,0.584192,186,85,20,101
1,I-NAME_STUDENT,0.756410,0.440299,0.556604,134,59,19,75
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,4,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,1,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [272]:
#Round 41: Try to replicate the previous results. Is one head better than 2?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 24,
     'Num Transformer Heads' : 1,
     'ff_dim' : 48,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 3000
TEST_SIZE = 0.3

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 119s 270ms/step - loss: 0.3276
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 184ms/step - loss: 0.0615
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 77s 196ms/step - loss: 0.0201


In [273]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 15952 tokens
True Positive: 190, False Positive: 95, False Negative: 166
Weighted Precision: 0.6107788621066405
Weighted Recall: 0.5337078651685394
Weighted F1: 0.5639315188147174
Micro F5: 0.537833424060969


2024-05-20 13:30:51.260969: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.713235,0.521505,0.602484,186,97,39,89
1,I-NAME_STUDENT,0.632653,0.694030,0.661922,134,93,54,41
2,B-EMAIL,0.000000,0.000000,0.000000,2,0,0,2
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,10,0,2,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [275]:
#Round 42: Vary the sample size balance with negative classes
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 116s 324ms/step - loss: 0.3795
Epoch 2/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 57s 188ms/step - loss: 0.0699
Epoch 3/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 57s 187ms/step - loss: 0.0237


In [276]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 156, False Positive: 41, False Negative: 260
Weighted Precision: 0.7762579419105967
Weighted Recall: 0.375
Weighted F1: 0.503248371998372
Micro F5: 0.38274983485892233


2024-05-20 13:40:07.968210: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.849558,0.454976,0.592593,211,96,17,115
1,I-NAME_STUDENT,0.855072,0.366460,0.513043,161,59,10,102
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,16,0,13,16
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.500000,0.083333,0.142857,12,1,1,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [277]:
#Round 43: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 117s 324ms/step - loss: 0.4000
Epoch 2/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 61s 200ms/step - loss: 0.0724
Epoch 3/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 59s 194ms/step - loss: 0.0223


In [278]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 240, False Positive: 108, False Negative: 176
Weighted Precision: 0.658355241959107
Weighted Recall: 0.5769230769230769
Weighted F1: 0.6103073021974821
Micro F5: 0.5805731298846297


2024-05-20 13:45:13.223341: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.717822,0.687204,0.702179,211,145,57,66
1,I-NAME_STUDENT,0.723077,0.583851,0.646048,161,94,36,67
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.000000,0.000000,0.000000,16,0,14,16
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.500000,0.083333,0.142857,12,1,1,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [279]:
#Round 44: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 131s 346ms/step - loss: 0.3666
Epoch 2/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 59s 193ms/step - loss: 0.0702
Epoch 3/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 60s 197ms/step - loss: 0.0227


In [280]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 198, False Positive: 104, False Negative: 218
Weighted Precision: 0.6398580260606971
Weighted Recall: 0.47596153846153844
Weighted F1: 0.5329221093925399
Micro F5: 0.48103158288170433


2024-05-20 13:50:20.039638: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.621359,0.606635,0.613909,211,128,78,83
1,I-NAME_STUDENT,0.831325,0.428571,0.565574,161,69,14,92
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.076923,0.062500,0.068966,16,1,12,15
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [281]:
#Round 45: One or two heads?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 144s 383ms/step - loss: 0.4084
Epoch 2/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 64s 208ms/step - loss: 0.0723
Epoch 3/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 63s 208ms/step - loss: 0.0235


In [282]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 193, False Positive: 193, False Negative: 223
Weighted Precision: 0.5706435630981149
Weighted Recall: 0.46394230769230765
Weighted F1: 0.5034664568080774
Micro F5: 0.46523270906730946


2024-05-20 13:55:52.239681: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.676056,0.454976,0.543909,211,96,46,115
1,I-NAME_STUDENT,0.573248,0.559006,0.566038,161,90,67,71
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,1,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.084746,0.312500,0.133333,16,5,54,11
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.090909,0.166667,0.117647,12,2,20,10
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,1,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [283]:
#Round 46: One or two heads?
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 8}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 145s 414ms/step - loss: 0.5079
Epoch 2/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 67s 219ms/step - loss: 0.0854
Epoch 3/3
304/304 ━━━━━━━━━━━━━━━━━━━━ 65s 213ms/step - loss: 0.0286


In [284]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 180, False Positive: 56, False Negative: 236
Weighted Precision: 0.7645285868584721
Weighted Recall: 0.4326923076923076
Weighted F1: 0.5454857526828225
Micro F5: 0.4400150432493419


2024-05-20 14:01:45.063153: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.864407,0.483412,0.620061,211,102,16,109
1,I-NAME_STUDENT,0.715686,0.453416,0.555133,161,73,29,88
2,B-EMAIL,1.000000,0.200000,0.333333,5,1,0,4
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.214286,0.187500,0.200000,16,3,11,13
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,1.000000,0.083333,0.153846,12,1,0,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [285]:
#Round 47: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 64,
     'Batch Size' : 16}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
152/152 ━━━━━━━━━━━━━━━━━━━━ 118s 607ms/step - loss: 0.7584
Epoch 2/3
152/152 ━━━━━━━━━━━━━━━━━━━━ 36s 239ms/step - loss: 0.1247
Epoch 3/3
152/152 ━━━━━━━━━━━━━━━━━━━━ 36s 239ms/step - loss: 0.0384


In [286]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 154, False Positive: 41, False Negative: 262
Weighted Precision: 0.740625
Weighted Recall: 0.3701923076923077
Weighted F1: 0.4931407728043678
Micro F5: 0.37791411042944784


2024-05-20 14:11:48.687259: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.864865,0.454976,0.596273,211,96,15,115
1,I-NAME_STUDENT,0.770270,0.354037,0.485106,161,57,17,104
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.100000,0.062500,0.076923,16,1,9,15
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.000000,0.000000,0.000000,12,0,0,12
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [287]:
record_df

,Model Description,Weighted Precision,Weighted Recall,Weighted F1,Micro F5,True Positive,False Negative,False Positive,Sample Size,Epochs,Sample Random State,Batch Size,Num Transformer Heads,ff_dim,embed_dim,Dataset Type
0,NER Transformer Without LSTM,0.31696,0.325714,0.301969,0.324929,114,236,258,15562,5,35,32,4,64,32,validation
1,NER Transformer Without LSTM,0.370838,0.548571,0.435372,0.538686,192,158,325,15562,5,35,16,4,128,64,validation
2,NER Transformer With Bidirectional LSTM,0.753249,0.388571,0.508894,0.39659,136,214,30,15562,5,35,16,4,128,64,validation
3,NER Transformer Without LSTM,0.448278,0.502857,0.425508,0.494542,176,174,327,15562,5,35,16,6,128,64,validation
4,NER Transformer With Bidirectional LSTM,0.750495,0.365714,0.481064,0.373639,128,222,29,15562,5,35,16,2,128,64,validation
5,NER Transformer With Bidirectional LSTM,0.810733,0.4,0.534415,0.40853,140,210,20,15562,3,35,16,2,128,64,validation
6,NER Transformer With Bidirectional LSTM,0.57781,0.56,0.568218,0.562783,196,154,109,15562,3,35,8,2,32,16,validation
7,NER Transformer With Bidirectional LSTM,0.692859,0.454286,0.54621,0.461178,159,191,55,15562,3,35,8,2,64,32,validation
8,NER Transformer Without LSTM,0.506493,0.405714,0.4365,0.409222,142,208,130,15562,3,35,8,2,128,64,validation
9,NER Transformer Without LSTM,0.422136,0.422857,0.408398,0.424771,148,202,161,15562,3,35,8,4,128,64,validation


In [288]:
#Round 48: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 234s 341ms/step - loss: 0.2319
Epoch 2/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 136s 223ms/step - loss: 0.0532
Epoch 3/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 135s 223ms/step - loss: 0.0163


In [289]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 228, False Positive: 194, False Negative: 188
Weighted Precision: 0.60238110466769
Weighted Recall: 0.548076923076923
Weighted F1: 0.5389779098952171
Micro F5: 0.5477730548881907


2024-05-20 14:35:18.954069: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.515152,0.725118,0.602362,211,153,144,58
1,I-NAME_STUDENT,0.853659,0.434783,0.576132,161,70,12,91
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,2,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.090909,0.187500,0.122449,16,3,30,13
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.250000,0.166667,0.200000,12,2,6,10
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [290]:
#Round 49: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 1,
     'ff_dim' : 63,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 236s 347ms/step - loss: 0.3423
Epoch 2/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 144s 237ms/step - loss: 0.0575
Epoch 3/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 144s 238ms/step - loss: 0.0179


In [291]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 195, False Positive: 134, False Negative: 221
Weighted Precision: 0.7769946865150353
Weighted Recall: 0.46874999999999994
Weighted F1: 0.5762422464325414
Micro F5: 0.47255102991891135


2024-05-20 14:46:43.486527: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.829457,0.507109,0.629412,211,107,22,104
1,I-NAME_STUDENT,0.900000,0.503106,0.645418,161,81,9,80
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.057143,0.375000,0.099174,16,6,99,10
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.200000,0.083333,0.117647,12,1,4,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [292]:
#Round 49: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 32,
     'Num Transformer Heads' : 2,
     'ff_dim' : 64,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 230s 337ms/step - loss: 0.3825
Epoch 2/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 148s 243ms/step - loss: 0.0651
Epoch 3/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 144s 237ms/step - loss: 0.0163


In [293]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 227, False Positive: 107, False Negative: 189
Weighted Precision: 0.7068307434309755
Weighted Recall: 0.5456730769230769
Weighted F1: 0.6136350788319291
Micro F5: 0.5498416247438047


2024-05-20 15:01:39.770660: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.758824,0.611374,0.677165,211,129,41,82
1,I-NAME_STUDENT,0.807018,0.571429,0.669091,161,92,22,69
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.125000,0.250000,0.166667,16,4,28,12
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.166667,0.166667,0.166667,12,2,10,10
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [294]:
#Round 49: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 24,
     'Num Transformer Heads' : 2,
     'ff_dim' : 48,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

Epoch 1/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 257s 380ms/step - loss: 0.3557
Epoch 2/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 145s 239ms/step - loss: 0.0662
Epoch 3/3
607/607 ━━━━━━━━━━━━━━━━━━━━ 176s 290ms/step - loss: 0.0214


In [295]:
results, record_dict = calculate_metrics(val_dataset, ner_model)
r.update({'Dataset Type': 'validation'})
r.update(record_dict)
record_df.add_record(r)
results

processed 11623 tokens
True Positive: 237, False Positive: 103, False Negative: 179
Weighted Precision: 0.6760261797026501
Weighted Recall: 0.5697115384615384
Weighted F1: 0.6095086299569995
Micro F5: 0.5737430167597766


2024-05-20 15:26:32.092252: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,labels,precision,recall,f1,total,TP,FP,FN
0,B-NAME_STUDENT,0.639810,0.639810,0.639810,211,135,76,76
1,I-NAME_STUDENT,0.831933,0.614907,0.707143,161,99,20,62
2,B-EMAIL,0.000000,0.000000,0.000000,5,0,0,5
3,I-EMAIL,0.000000,0.000000,0.000000,0,0,0,0
4,B-URL_PERSONAL,0.142857,0.062500,0.086957,16,1,6,15
5,I-URL_PERSONAL,0.000000,0.000000,0.000000,0,0,0,0
6,B-ID_NUM,0.500000,0.083333,0.142857,12,1,1,11
7,I-ID_NUM,0.000000,0.000000,0.000000,0,0,0,0
8,B-USERNAME,0.000000,0.000000,0.000000,2,0,0,2
9,I-USERNAME,0.000000,0.000000,0.000000,0,0,0,0


In [4]:
params = {'embed_dim' : [24, 32],
          'ff_dim' : [48, 64,96],
          'Batch Size' : [4, 8, 16},
          'Num Transformer Heads' : [1, 2, 4, 8],
          'Sample Random State': 72}

NUM_NEG_SAMPLES = 1500
TEST_SIZE = 0.2
results_df = RecordDataFrame()
import itertools

dim_combos = set(list(itertools.combinations(itertools.chain(params['embed_dim'], params['ff_dim']),2)))
results_list = []

dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, params['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']

#'bidirectional_lstm3.csv'
#'individual_layer_results4.csv'
#'bidirectional_lstm4.csv' - early stopping implemented
#'individual_layer_results5.csv'

In [23]:
del results_df
del results_list

In [24]:
results_df = RecordDataFrame()
results_list = []

In [ ]:
batch = 16
train_dataset, val_dataset = get_training_data(X,y,state = params['Sample Random State'], batch_size = batch)

for num in params['Num Transformer Heads'][1:]:
    for (embed_dim, ff_dim) in list(dim_combos):
        for i in range(3):

            r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
                 'Sample Random State' : params['Sample Random State'],
                 'Epochs' : 10,
                 'embed_dim' : embed_dim,
                 'Num Transformer Heads' : num,
                 'ff_dim' : ff_dim,
                 'Batch Size' : batch}

            print(r)
            early_stopping = tf.keras.callbacks.EarlyStopping(mode = 'min', patience = 2)
            ner_model = NERModelWithLSTM(num_tags, 
                                              vocab_size, 
                                              embed_dim=r['embed_dim'], 
                                              num_heads=r['Num Transformer Heads'], 
                                              ff_dim=r['ff_dim'])
            ner_model.compile(optimizer='adam',loss=loss)
            ner_model.fit(train_dataset, epochs=r['Epochs'],validation_data = val_dataset, callbacks = early_stopping)
            tmp_results, record_dict = calculate_metrics(val_dataset, ner_model)
            r.update({'Dataset Type': 'validation'})
            r.update(record_dict)
            results_df.add_record(r)
            results_df.to_csv('bidirectional_lstm5.csv')
            results_list.append(tmp_results)
            pd.concat(results_list,axis = 0,ignore_index=True).to_csv('individual_layer_results6.csv')
            micro_f5 = record_dict['Micro F5']
            if micro_f5 > 0.7:
                test_dataset = get_test_data(train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)), 'tokens'],
                             train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)),'labels'], batch)
                tmp2_results, record_dict = calculate_metrics(test_dataset, ner_model)
                r.update({'Dataset Type': 'test'})
                r.update(record_dict)
                results_df.add_record(r)
                results_df.to_csv('bidirectional_lstm5.csv')
                results_list.append(tmp2_results)
                pd.concat(results_list,axis = 0,ignore_index=True).to_csv('individual_layer_results6.csv')

{'Model Description': 'NER Transformer With Bidirectional LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


2024-05-21 14:24:52.316644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


127/127 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - loss: 0.8467 - val_loss: 0.1522
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - loss: 0.1660 - val_loss: 0.1050
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - loss: 0.0751 - val_loss: 0.0985
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 12s 98ms/step - loss: 0.0276 - val_loss: 0.0779
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - loss: 0.0142 - val_loss: 0.0822
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - loss: 0.0073 - val_loss: 0.0885
processed 9843 tokens
True Positive: 232, False Positive: 77, False Negative: 128
Weighted Precision: 0.7319097790507365
Weighted Recall: 0.6444444444444444
Weighted F1: 0.6794771000852334
Micro F5: 0.6479750778816199
{'Model Description': 'NER Transformer With Bidirectional LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 24, 'Num Transformer Heads': 2, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


2024-05-21 14:26:10.976771: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 16s 108ms/step - loss: 0.5988 - val_loss: 0.1448
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - loss: 0.1389 - val_loss: 0.1035
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - loss: 0.0538 - val_loss: 0.0715
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 13s 101ms/step - loss: 0.0219 - val_loss: 0.0727
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - loss: 0.0123 - val_loss: 0.0734


...

127/127 ━━━━━━━━━━━━━━━━━━━━ 131s 841ms/step - loss: 0.6781 - val_loss: 0.1494
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 39s 310ms/step - loss: 0.1594 - val_loss: 0.1112
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 38s 299ms/step - loss: 0.0583 - val_loss: 0.0987
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 37s 294ms/step - loss: 0.0167 - val_loss: 0.0970
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 39s 304ms/step - loss: 0.0060 - val_loss: 0.0976
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 38s 301ms/step - loss: 0.0028 - val_loss: 0.0991
processed 9843 tokens
True Positive: 189, False Positive: 95, False Negative: 171
Weighted Precision: 0.747327774303384
Weighted Recall: 0.525
Weighted F1: 0.6051119672788161
Micro F5: 0.5292977165015079
{'Model Description': 'NER Transformer With Bidirectional LSTM', 'Sample Random State': 72, 'Epochs': 10, 'embed_dim': 32, 'Num Transformer Heads': 4, 'ff_dim': 96, 'Batch Size': 16}
Epoch 1/10


2024-05-21 16:42:00.989480: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


127/127 ━━━━━━━━━━━━━━━━━━━━ 194s 459ms/step - loss: 0.5745 - val_loss: 0.1499
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 197ms/step - loss: 0.1545 - val_loss: 0.1308
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 194ms/step - loss: 0.0593 - val_loss: 0.0724
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 193ms/step - loss: 0.0158 - val_loss: 0.0756
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 24s 193ms/step - loss: 0.0099 - val_loss: 0.0855


2024-05-21 16:47:00.126797: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


processed 9843 tokens
True Positive: 270, False Positive: 156, False Negative: 90
Weighted Precision: 0.6675203501431549
Weighted Recall: 0.75
Weighted F1: 0.7036054973367329
Micro F5: 0.7447485677912158


In [6]:
test_dataset = get_test_data(train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)), 'tokens'],
                             train.loc[train.document.isin(list(df1.loc[list(test_doc_ids),:].document)),'labels'], batch)
tmp2_results, record_dict = calculate_metrics(test_dataset, ner_model)
r.update({'Dataset Type': 'test'})
r.update(record_dict)
results_df.add_record(r)
results_df.to_csv('bidirectional_lstm3.csv')
results_list.append(tmp2_results)
pd.concat(results_list,axis = 0,ignore_index=True).to_csv('individual_layer_results4.csv')

processed 23458 tokens
True Positive: 29, False Positive: 39, False Negative: 37
Weighted Precision: 0.5443031715758988
Weighted Recall: 0.43939393939393934
Weighted F1: 0.46230158730158727
Micro F5: 0.43888242142025613


2024-05-21 11:36:17.307906: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
#Round 50: Try to replicate with the same dataset
r = {'Model Description' : 'NER Transformer With Bidirectional LSTM',
     'Sample Random State' : 42,
     'Epochs' : 3,
     'embed_dim' : 24,
     'Num Transformer Heads' : 2,
     'ff_dim' : 48,
     'Batch Size' : 4}

NUM_NEG_SAMPLES = 2000
TEST_SIZE = 0.2

#dataset, test_doc_ids = create_sample_data(df1, TEST_SIZE, NUM_NEG_SAMPLES, r['Sample Random State'])
X = dataset['tokenized_sentence']
y = dataset['labels_in_sentence']
train_dataset, val_dataset = get_training_data(X,y,state = r['Sample Random State'], batch_size = r['Batch Size'])

ner_model = NERModelWithLSTM(num_tags, vocab_size, embed_dim=r['embed_dim'], num_heads=r['Num Transformer Heads'], ff_dim=r['ff_dim'])
ner_model.compile(optimizer='adam',loss=loss)
ner_model.fit(train_dataset, epochs=r['Epochs'])

In [298]:
record_df.to_csv('recorded_runs.csv', index = False)

**Using the full text instead of the sentence context for training dataset**

In [171]:
X_train, X_test, y_train, y_test = train_test_split(train['tokens'],train['labels'],test_size = 0.2, random_state = 42)

X_train = pad_sequences(X_train.map(lookup_layer))
X_test = pad_sequences(X_test.map(lookup_layer))
y_train = pad_sequences(pd.Series([[encoding[r] for r in row] for row in y_train]))
y_test = pad_sequences(pd.Series([[encoding[r] for r in row] for row in y_test]))

batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)

ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)
ner_model.compile(optimizer='adam',loss=loss)

ner_model.fit(train_dataset, epochs=5)

Epoch 1/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 1402s 8s/step - loss: 0.1482
Epoch 2/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 1367s 8s/step - loss: 0.0064
Epoch 3/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 1315s 8s/step - loss: 0.0044
Epoch 4/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 1325s 8s/step - loss: 0.0017
Epoch 5/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 1336s 8s/step - loss: 0.0014


Epoch 1/5
loss: 0.1482
Epoch 2/5
loss: 0.0064
Epoch 3/5
loss: 0.0044
Epoch 4/5
loss: 0.0017
Epoch 5/5
loss: 0.0014

In [206]:
real, preds = calculate_metrics(test_dataset)

2024-05-17 02:31:25.141757: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


processed 1005699 tokens
True Positive: 107, True Negative: 1004925, False Positive: 210, False Negative: 416
41 tokens identified as PII but mislabelled.
Accuracy (non-O): 0.18971631205673758
Accuracy: 0.9993367796925322
Precision: 0.33753943217665616
Recall: 0.2045889101338432
F1 score: 0.12738095238095237
S-micro score: 0.20773596176821985


In [204]:
results = calculate_metrics_by_label(real, preds, labels)

True Positive: 107, False Positive: 251, False Negative: 457


/var/folders/5f/zt7yqxtn7wzf5xtwg4zjzfn80000gn/T/ipykernel_1813/1002851054.py:18: RuntimeWarning: invalid value encountered in scalar divide
  p = tp / (tp + fp)
/var/folders/5f/zt7yqxtn7wzf5xtwg4zjzfn80000gn/T/ipykernel_1813/1002851054.py:19: RuntimeWarning: invalid value encountered in scalar divide
  r = tp / (tp + fn)
/var/folders/5f/zt7yqxtn7wzf5xtwg4zjzfn80000gn/T/ipykernel_1813/1002851054.py:21: RuntimeWarning: invalid value encountered in scalar divide
  s = (1+(beta**2))*tp/(((1+(beta**2))*tp) + ((beta**2)*fn) + fp)


In [202]:
results

,labels,precision,recall,f1 score,s_micro
0,B-NAME_STUDENT,0.294118,0.266160,0.139721,0.267136
1,I-NAME_STUDENT,0.308333,0.151639,0.101648,0.154662
2,B-EMAIL,NaN,0.000000,NaN,0.000000
3,I-EMAIL,NaN,NaN,NaN,NaN
4,B-URL_PERSONAL,NaN,0.000000,NaN,0.000000
5,I-URL_PERSONAL,NaN,NaN,NaN,NaN
6,B-ID_NUM,NaN,0.000000,NaN,0.000000
7,I-ID_NUM,NaN,NaN,NaN,NaN
8,B-USERNAME,NaN,NaN,NaN,NaN
9,I-USERNAME,NaN,NaN,NaN,NaN
